<a href="https://colab.research.google.com/github/CsonVass/lpr-image-processing-hw/blob/main/sniper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Only run first time, to set up the environment!

In [ ]:
# mount google drive with essential files
from google.colab import drive
drive.mount('/content/drive')
%cd ..

In [ ]:
# install needed modules
!pip install opencv-python
!pip install numpy
!pip install matplotlib
!pip install pillow

In [ ]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageFilter
from google.colab.patches import cv2_imshow

In [ ]:
# create a directory where the results will be saved
!mkdir detections 
!ls

In [ ]:
# the class that utilises the pre-trained weights
class LicensePlateDetector:
    def __init__(self, pth_weights: str, pth_cfg: str, pth_classes: str):
        self.net = cv2.dnn.readNet(pth_weights, pth_cfg)
        self.classes = []
        with open(pth_classes, 'r') as f:
            self.classes = f.read().splitlines()
        self.font = cv2.FONT_HERSHEY_PLAIN
        self.color = (255, 0, 0)
        self.coordinates = []
        self.confidences = []
        self.img = None
        self.fig_image = None
        self.roi_image = []
        
    # detects all the bounding boxes  
    def detect(self, img_path: str):
        orig = cv2.imread(img_path)
        self.img = orig
        img = orig.copy()
        height, width, _ = img.shape
        blob = cv2.dnn.blobFromImage(img, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
        self.net.setInput(blob)
        output_layer_names = self.net.getUnconnectedOutLayersNames()
        layer_outputs = self.net.forward(output_layer_names)
        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores) 
                confidence = scores[class_id]
                if confidence > 0.82:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append((float(confidence)))
                    class_ids.append(class_id)
        self.confidences = confidences
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                self.coordinates.append([x, y, w, h])
        return
    
    
    # crops all the detected license plates
    def crop_plate(self):
        for plate in range(len(self.coordinates)):
          x, y, w, h = self.coordinates[plate]
          self.roi_image.append(self.img[y:y + h, x:x + w])
        return


## Choose the folder where the images are located

In [ ]:
image_folder = '/content/drive/MyDrive/images/'

In [ ]:
for image in os.listdir(image_folder):
  if image == 'sniped':
    continue

  image_path = os.path.join(image_folder, image)

  image_name = image.split('.')[0]
  # print("image_name: \t\t" + image_name)

  new_folder = os.path.join('/detections/', image_name)
  # print("new_folder: \t\t" + new_folder)

  drive_new_folder = os.path.join('/content/drive/MyDrive/images/sniped/', image_name)
  print("drive_new_folder: " + drive_new_folder)

  lpd = LicensePlateDetector(
      pth_weights='/content/drive/MyDrive/yolov4/backup/yolov4-obj_last.weights',
      pth_cfg='/content/drive/MyDrive/yolov4/yolov4-obj.cfg', 
      pth_classes='/content/drive/MyDrive/yolov4/obj.names')

  # Detect license plate
  lpd.detect(image_path)

  # Crop plate and show cropped plate
  lpd.crop_plate()
  for i in range(len(lpd.roi_image)):
    print("License plate no. {" + str(i + 1) + "} with confidence: " + str(lpd.confidences[i]))

  print("")

  if not os.path.exists(new_folder) and not os.path.exists(drive_new_folder):

    os.mkdir(new_folder)
    os.mkdir(drive_new_folder)

    for plate in range(len(lpd.roi_image)):

      im = Image.fromarray(lpd.roi_image[plate])

      # scale up
      size = 416, int((416 / im.size[0]) * im.size[1])
      im = im.resize(size)

      # little gaussian blur to filter noise
      im = im.filter(ImageFilter.GaussianBlur(radius = 1))

      # save to drive and cloud env
      im.save(new_folder + '/' + str(plate) + image)
      im.save(drive_new_folder + '/' + str(plate) + image)

      # display
      # plt.figure(figsize=(10, 4))
      # plt.imshow(cv2.cvtColor(lpd.roi_image[plate], cv2.COLOR_BGR2RGB))

drive_new_folder: /content/drive/MyDrive/images/sniped/29424b7a1926ec4e
License plate no. {1} with confidence: 0.9513380527496338

drive_new_folder: /content/drive/MyDrive/images/sniped/18666381
License plate no. {1} with confidence: 0.9583039879798889

drive_new_folder: /content/drive/MyDrive/images/sniped/18664994
License plate no. {1} with confidence: 0.875501811504364

drive_new_folder: /content/drive/MyDrive/images/sniped/RYX-928_h
License plate no. {1} with confidence: 0.8943033814430237
License plate no. {2} with confidence: 0.8404107093811035

drive_new_folder: /content/drive/MyDrive/images/sniped/SWS-448_h
License plate no. {1} with confidence: 0.949833333492279

drive_new_folder: /content/drive/MyDrive/images/sniped/316476175_650984866763427_8310885032851288955_n
License plate no. {1} with confidence: 0.83013516664505

drive_new_folder: /content/drive/MyDrive/images/sniped/20434076
License plate no. {1} with confidence: 0.945834755897522
License plate no. {2} with confidence:

In [ ]:
im = cv2.imread("/content/drive/MyDrive/images/sniped/20430001/020430001.jpg")
cv2_imshow(im)
thresh = 127
im_bw = cv2.threshold(im, thresh, 255, cv2.THRESH_BINARY)[1]
cv2_imshow(im_bw)

## Run this box to purge all the saved files!

In [ ]:
# PURGE !!
!rm -f -R /content/drive/MyDrive/images/sniped/*
!rm -f -R detections/*

In [ ]:
!ls /content/drive/MyDrive/images/sniped/
!ls detections/